In [ ]:
# Clear cache
arcpy.management.Delete("in_memory")
arcpy.management.Delete("memory")
# Set the workspace (working directory)
arcpy.env.workspace = r"F:\LewisK\PPR_GovType\PPR_GovType.gdb"
arcpy.env.overwriteOutput = True
# Import necessary Libraries
from dateutil.relativedelta import relativedelta
import datetime
from arcgis.gis import GIS
import arcpy
import os
from arcgis.features import FeatureLayer
from arcgis.features import FeatureLayerCollection
import pandas as pd
import math
#Coverage by governance type
# Identify the month and year of update
date = datetime.datetime.today()
month = date.strftime("%b")
year = date.strftime("%Y")
# Define Mollweide projection
mollweide_sr = arcpy.SpatialReference(54009)
# Define dataset paths in a list of tuples, where each tuple has a dataset name and the dataset path
datasets = [
     ('wdpa2024', r'O:/PP2024/gov_type_flat.gdb/wdpa_poly_point_public_restricted_basemap_intersect_statusyr2024_Aug2024_GOV_TYPE_union'),
     ('wdpa2022', r'O:/PP2024/gov_type_flat.gdb/wdpa_poly_point_public_restricted_basemap_intersect_statusyr2022_Aug2024_GOV_TYPE_union'),
    ('wdpa2020', r'O:/PP2024/gov_type_flat.gdb/wdpa_poly_point_public_restricted_basemap_intersect_statusyr2020_Aug2024_GOV_TYPE_union)',
    ('wdoecm2024', r'O:/PP2024/gov_type_flat.gdb/wdoecm_poly_point_basemap_intersect_statusyr2024_Aug2024_GOV_TYPE_union'),
    ('wdoecm2022', r'O:/PP2024/gov_type_flat.gdb/wdoecm_poly_point_basemap_intersect_statusyr2022_Aug2024_GOV_TYPE_union'),
    ('wdoecm2020', r'O:/PP2024/gov_type_flat.gdb/wdoecm_poly_point_basemap_intersect_statusyr2020_Aug2024_GOV_TYPE_union')
]
print("Dataset paths initialized")
# Load basemap and repair geometry
basemap = r'O:/PP2024/basemap.gdb/ungeo_eez_gie_national_unsd49regions_v2'
arcpy.management.MakeFeatureLayer(basemap, "basemap2")
arcpy.management.RepairGeometry("basemap2", "DELETE_NULL", "OGC")
print("Basemap loaded and repaired")
# Loop through each dataset
for dataset_name, dataset_path in datasets:
    print(f"Processing {dataset_name}")
    # Step 1: Load dataset into memory and convert multipart to singlepart
    arcpy.management.MakeFeatureLayer(dataset_path, f"{dataset_name}_mp")
    arcpy.management.MultipartToSinglepart(f"{dataset_name}_mp", f"{dataset_name}_sp")
    print(f"{dataset_name}: Multipart converted to Singlepart")
    # Step 2: Dice the dataset
    arcpy.management.Dice(f"{dataset_name}_sp", f"{dataset_name}_dice", vertex_limit="5000")
    print(f"{dataset_name}: Singlepart features diced")
    # Step 3: Repair geometry
    arcpy.management.RepairGeometry(f"{dataset_name}_dice", "DELETE_NULL", "OGC")
    print(f"{dataset_name}: Geometry repaired")
    # Step 4: Intersect with basemap
    arcpy.analysis.PairwiseIntersect(
        in_features=[f"{dataset_name}_dice", "basemap2"],
        out_feature_class=f"{dataset_name}_dice_intersect",
        join_attributes="ALL",
        output_type="INPUT"
    )
    print(f"{dataset_name}: Intersected with basemap")
    # Step 5: Repair geometry again (after intersect)
    arcpy.management.RepairGeometry(f"{dataset_name}_dice_intersect", "DELETE_NULL", "OGC")
    print(f"{dataset_name}: Post-intersect geometry repaired")
    # Step 6: Project to Mollweide
    arcpy.management.Project(
        in_dataset=f"{dataset_name}_dice_intersect",
        out_dataset=f"{dataset_name}_dice_intersect_mollweide",
        out_coor_system=mollweide_sr
    )
    print(f"{dataset_name}: Projected to Mollweide")
    # Step 7: Repair geometry one last time (after projection)
    arcpy.management.RepairGeometry(f"{dataset_name}_dice_intersect_mollweide", "DELETE_NULL", "OGC")
    print(f"{dataset_name}: Post-projection geometry repaired")
    # Step 8: Calculate geometry attributes
    arcpy.management.CalculateGeometryAttributes(
        f"{dataset_name}_dice_intersect_mollweide",
        [["PA_AREA", "AREA_GEODESIC"]],
        area_unit="SQUARE_KILOMETERS",
        coordinate_system=mollweide_sr
    )
    print(f"{dataset_name}: Geometry attributes calculated")
    # Step 9: Perform Summary Statistics
    # 9.1 Summing PA_AREA by GOV_TYPE
    govtype_sum_table = f'F:\\LewisK\PPR_GovType\\PPR_GovType.gdb\\{dataset_name}_govtype_sum_table'
    arcpy.analysis.Statistics(
        in_table=f"{dataset_name}_dice_intersect_mollweide",
        out_table=govtype_sum_table,
        statistics_fields=[["PA_AREA", "SUM"]],
        case_field=["GOV_TYPE", "TYPE"]
        )
    print(f"{dataset_name}: PA_AREA summed by GOV_TYPE and exported to {govtype_sum_table}")
    # 9.2 Summing PA_AREA by GOV_TYPE and Sub Region Name
    govtype_subregion_sum_table = f'F:\\LewisK\PPR_GovType\\PPR_GovType.gdb\\{dataset_name}_govtype_subregion_sum_table'
    arcpy.analysis.Statistics(
        in_table=f"{dataset_name}_dice_intersect_mollweide",
        out_table=govtype_subregion_sum_table,
        statistics_fields=[["PA_AREA", "SUM"]],
        case_field=["GOV_TYPE", "Sub-region Name"]
    )
    print(f"{dataset_name}: PA_AREA summed by GOV_TYPE and Region Name, and exported to {govtype_subregion_sum_table}")
    # 9.3 Summing PA_AREA by GOV_TYPE and Region Name
    govtype_region_sum_table = f'F:\\LewisK\PPR_GovType\\PPR_GovType.gdb\\{dataset_name}_govtype_region_sum_table'
    arcpy.analysis.Statistics(
        in_table=f"{dataset_name}_dice_intersect_mollweide",
        out_table=govtype_region_sum_table,
        statistics_fields=[["PA_AREA", "SUM"]],
        case_field=["GOV_TYPE", "Region Name"]
        )
    print(f"{dataset_name}: PA_AREA summed by GOV_TYPE and Region Name, and exported to {govtype_region_sum_table}")
print("All datasets processed successfully")